In [1]:
import numpy as np

In [34]:
test1 = {'input':[[1,7,4,2,1,3,11,5], 10], 'output':[4,2,1,3]}
test2 = {'input':[[10,10,10,5,5], 10], 'output':[10]}
test3 = {'input':[[1,2,3,4,5,6], 100], 'output':None}
test4 = {'input':[[1,2,4,9,1], 10], 'output':[9,1]}
test5 = {'input':[[], 10], 'output':None}

In [36]:
def find_subarray(array, total):
    idx = -1

    for _ in array:
        subarray = []
        sum = 0
        idx += 1

        for n in range(idx, len(array)):
            sum += array[n]
            if sum < total:
                subarray.append(array[n])
            elif sum == total:
                subarray.append(array[n])
                return subarray
            else:
                break
    
    return None

In [45]:
def find_subarray2(array, target):
    i, j, total = 0, 0, 0

    while i <= len(array) - 1 and j <= len(array):
        if sum(array[i:j]) < target:
            j += 1
        elif sum(array[i:j]) > target:
            i += 1
        else:
            return array[i:j]
    
    return None

In [46]:
find_subarray2(*test4['input'])

[9, 1]